In [8]:
import json
from requests import post, get
from credentials import *
import openai

In [9]:
openai.api_key = OPENAI_API_KEY
yan_tok = IAMTOKEN
giga_tok = GIGATOK

In [10]:
instructions = json.loads(open('instructions.json').read())

In [11]:
print(instructions)

{'1': {'instruction': 'Опиши процесс выбора кандидата на вакансию.', 'type': 'hr', 'Yandex_GPT': {'Response': 'Процесс выбора кандидата на вакантное место представляет собой сложную и многогранную задачу, которая требует тщательного анализа и оценки потенциальных кандидатов. Этот процесс обычно включает в себя следующие шаги:\n\n1. Определение требований к кандидату: Прежде всего, необходимо четко определить требования к кандидату на вакантную должность. Это может включать в себя профессиональные навыки, опыт работы, образование, личные качества и другие аспекты, которые важны для успешного выполнения работы.\n\n2. Поиск кандидатов: После определения требований, можно начать поиск кандидатов. Это может быть осуществлено через различные каналы, такие как интернет-ресурсы, профессиональные сети, кадровые агентства или напрямую от работодателей.\n\n3. Оценка кандидатов: Проведя предварительный отбор, следует провести более тщательную оценку кандидатов. Для этого можно использовать различн

In [12]:
def make_request_yandex(instruction = None, model='general', tok = yan_tok):
    if type(instruction) != str:
        raise TypeError('instruction should be a string')
    url = "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct"
    headers = {
        "Authorization": f"Bearer {tok}",
        "x-folder-id": "b1gmdp3kcenod48c8ts0",
        "Content-Type": "application/json"
    }
    body = {"model": model,
            "generationOptions": {
                "partialResults": False,
                "temperature": 0.5,
                "maxTokens": 4096,
                },
            "instructionText": instruction,
            }
    return post(url=url, headers=headers, json=body)


In [13]:
from time import sleep

In [14]:
yand_responses_dict = {}

In [15]:
for k in instructions.keys():
    if instructions[k]['Yandex_GPT']['Response'] != None:
        continue
    instruction = instructions[k]['instruction']
    response = make_request_yandex(instruction=instruction).json()
    yand_responses_dict[k] = response
    try:
        instructions[k]['Yandex_GPT']['Response'] = response['result']['alternatives'][0]['text']
    except:
        instructions[k]['Yandex_GPT']['Response'] = response['error']['message']
    sleep(1.1)
    


In [73]:
with open('instructions.json', mode='w', encoding='utf-8') as w:
    w.write(json.dumps(instructions))

In [12]:
# Yandex
# url = "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct"
# headers = {
#         "Authorization": "Bearer <токен>",
#         "x-folder-id": "b1gmdp3kcenod48c8ts0",
#         "Content-Type": "application/json"
#     }

# instructionText = "Ты — SimpleGPT, русскоязычный автоматический ассистент, созданный в компании Simple AI. Ты разговариваешь с людьми и помогаешь им."

In [ ]:
# Gigachat
# url = "https://beta.saluteai.sberdevices.ru/v1/chat/completions"
# headers = {
#         "Authorization": "Bearer <токен>",
#         "Content-Type": "application/json"
#     }

In [16]:
def make_request_gigachat(instruction=None, token=giga_tok, model='GigaChat:latest'):
    if type(instruction) != str:
        raise TypeError('Instruction should be in str format.')
    url = "https://beta.saluteai.sberdevices.ru/v1/chat/completions"
    headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        }
    body = {"model": model,
            'messages' : [
                {
                    'role' : 'user',
                    'content' : instruction
                }
            ],
            'temperature' : 0.7
            }
    return post(url=url, headers=headers, json=body)

In [24]:
gigachat_response_dict = {}

In [26]:
for k in instructions.keys():
    if instructions[k]['Gigachat']['Response'] != None:
        continue
    instruction = instructions[k]['instruction']
    response = make_request_gigachat(instruction=instruction).json()
    gigachat_response_dict[k] = response    
    instructions[k]['Gigachat']['Response'] = response['choices'][0]['message']['content']
    sleep(1)
    


In [29]:
with open('instructions.json', mode='w', encoding='utf-8') as w:
    w.write(json.dumps(instructions))